# [Day 16] Advanced Training: FastAPI & Pydantic 종합 (Mix, 중하)

**"기초 문법을 뼈대로, 실제 웹 프레임워크의 살을 붙이는 과정입니다."**

Basic 파트에서 연습했던 데이터 다루기 스킬을, 이제 실제 서버 통신 환경(FastAPI)과 데이터 검증(Pydantic), 그리고 객체지향(Class)에 적용해 봅니다. 난이도는 '중하'로, 부담 없이 구조를 익히는 데 집중합니다.

---

### Q1. Pydantic 모델 설계 (데이터 검증)

사용자 회원가입 시 받을 데이터를 정의하는 Pydantic 모델 `UserCreate`를 만들어주세요.

**요구사항:**
1. `username` (문자열, 필수)
2. `age` (정수, 필수)
3. `is_active` (불리언, 기본값 `True`)
4. `@field_validator('age')`를 사용하여 `age`가 0보다 커야 한다는 검증 로직을 추가하세요. (0 이하면 ValueError 발생)

In [2]:
from pydantic import BaseModel, field_validator, ValidationError

class UserCreate(BaseModel):
    username: str 
    age: int
    is_active: bool = True

    # 2. 여기에 age 검증 로직(validator)을 작성하세요
    @field_validator("age")
    @classmethod
    def check_age(cls, value):
        if value <= 0:
            raise ValueError("나이는 0보다 커야 합니다")
        return value
# 테스트
try:
    good_user = UserCreate(username="Alice", age=25)
    print("Q1 정상 유저:", good_user)
    bad_user = UserCreate(username="Bob", age=-5) # 여기서 에러가 나야 합니다.
except ValidationError as e:
    print("Q1 에러 캐치 성공:", e.errors()[0]['msg'])

Q1 정상 유저: username='Alice' age=25 is_active=True
Q1 에러 캐치 성공: Value error, 나이는 0보다 커야 합니다


### Q2. FastAPI 엔드포인트 구현 (POST)

Q1에서 만든 `UserCreate` 모델을 사용하여, 새로운 유저를 생성하는 API 엔드포인트를 만들어주세요.

**요구사항:**
1. `@app.post("/users/")` 데코레이터를 사용하세요.
2. 입력받은 `user` 데이터(UserCreate 타입)를 딕셔너리로 변환(`user.model_dump()`)합니다.
3. 변환된 딕셔너리에 임의의 `id` (예: 1)를 추가하여 반환하세요.

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.post("/users/")
def create_user(user: UserCreate):
    user_dict = user.model_dump()

    user_dict["id"] = 1
    
    return user_dict

# (참고) 실제 서버 테스트는 uvicorn이나 TestClient가 필요하므로, 여기서는 함수 구조를 맞추는 데 집중해 주세요!

### Q3. 간단한 Class 설계 (메모리 DB)

유저 데이터를 저장하고 관리할 간단한 메모리 DB 클래스 `UserRepository`를 만들어주세요.

**요구사항:**
1. `__init__` 메서드에서 빈 리스트 `self.db`를 초기화합니다.
2. `add_user(self, user_dict: dict)` 메서드: 딕셔너리 형태의 유저를 `self.db`에 추가합니다.
3. `get_users(self)` 메서드: 현재 저장된 모든 유저 리스트 `self.db`를 반환합니다.

In [3]:
class UserRepository:
    def __init__(self):
        self.db = []
    def add_user(self, user_dict: dict):
        self.db.append(user_dict)
    def get_users(self):
        return self.db
# 테스트
repo = UserRepository()
repo.add_user({"id": 1, "username": "Alice", "age": 25})
repo.add_user({"id": 2, "username": "Dave", "age": 30})

print("Q3 저장된 유저 목록:", repo.get_users())
# 예상 출력: [{'id': 1, 'username': 'Alice', 'age': 25}, {'id': 2, 'username': 'Dave', 'age': 30}]

Q3 저장된 유저 목록: [{'id': 1, 'username': 'Alice', 'age': 25}, {'id': 2, 'username': 'Dave', 'age': 30}]
